In [28]:
from selenium import webdriver  # Need To import webdriver from selenium package
from bs4 import BeautifulSoup as bs

In [38]:
def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_1'
    search_term = search_term.replace(' ','+')
    url = template.format(search_term)
    url = url + '&page={}'
    return url

In [42]:
url = get_url('ultrawide monitor')

In [43]:
print(url)

https://www.amazon.com/s?k=ultrawide+monitor&ref=nb_sb_noss_1


In [44]:
browser = webdriver.Chrome(executable_path='C:/Users/HP/chromedriver.exe')
browser.get(url)

In [45]:
soup = bs(browser.page_source, 'lxml')

In [48]:
items = soup.find_all('div',{'data-component-type':'s-search-result'})

In [49]:
len(items)

22

In [50]:
item = items[0]

In [51]:
atag = item.h2.a

In [52]:
print(atag)

<a class="a-link-normal a-text-normal" href="/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&amp;adId=A102645020ZZO08BEH16G&amp;url=%2FLG-34WK650-W-34-UltraWide-21%2Fdp%2FB078GSH1LV%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dultrawide%2Bmonitor%26qid%3D1616527957%26sr%3D8-1-spons%26psc%3D1&amp;qualifier=1616527957&amp;id=2370384781466426&amp;widgetName=sp_atf">
<span class="a-size-medium a-color-base a-text-normal" dir="auto">LG 34WK650-W 34" UltraWide 21:9 IPS Monitor with HDR10 and FreeSync (2018), Black/White</span>
</a>


In [53]:
description = atag.text.strip()

In [54]:
description

'LG 34WK650-W 34" UltraWide 21:9 IPS Monitor with HDR10 and FreeSync (2018), Black/White'

In [55]:
link = 'https://www.amazon.com' + atag.get('href')

In [57]:
print(link)

https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A102645020ZZO08BEH16G&url=%2FLG-34WK650-W-34-UltraWide-21%2Fdp%2FB078GSH1LV%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dultrawide%2Bmonitor%26qid%3D1616527957%26sr%3D8-1-spons%26psc%3D1&qualifier=1616527957&id=2370384781466426&widgetName=sp_atf


In [58]:
price = item.find('span','a-price')

In [59]:
price_item = price.find('span','a-offscreen').text
price_item

'$296.99'

In [62]:
rating = item.i.text

In [66]:
rating.split()[0]

'4.7'

In [67]:
review_count = item.find('span',{'class':'a-size-base','dir':'auto'}).text

'910'

In [82]:
def extract_item(item):
    atag = item.h2.a
    description = atag.text.strip()
    
    link = 'https://www.amazon.com' + atag.get('href')
    
    try:
        price = item.find('span','a-price')
        price_item = price.find('span','a-offscreen').text
    except AttributeError:
        return
    
    try: 
        rating = item.i.text
        review_count = item.find('span',{'class':'a-size-base','dir':'auto'}).text
    except AttributeError:
        rating=''
        review_count=''
        
    result = (description, price_item, rating.split()[0], review_count, link)
    return result

In [83]:
records=[]
for item in items:
    #print(item)
    record = extract_item(item)
    if record:
        #print(record)
        records.append(record)

In [84]:
records[0]

('LG 34WK650-W 34" UltraWide 21:9 IPS Monitor with HDR10 and FreeSync (2018), Black/White',
 '$296.99',
 '4.7',
 '910',
 'https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A102645020ZZO08BEH16G&url=%2FLG-34WK650-W-34-UltraWide-21%2Fdp%2FB078GSH1LV%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dultrawide%2Bmonitor%26qid%3D1616527957%26sr%3D8-1-spons%26psc%3D1&qualifier=1616527957&id=2370384781466426&widgetName=sp_atf')

In [86]:
for row in records:
    print(row[1])

$296.99
$226.99
$549.99
$296.99
$449.99
$429.97
$799.99
$349.99
$30.95
$449.99
$549.99
$296.99
$208.99
$479.99
$429.99
$449.99
$59.98
$159.99
$209.99


## Putting All Together

In [87]:
import csv

In [101]:
from selenium import webdriver  # Need To import webdriver from selenium package
from bs4 import BeautifulSoup as bs

##geting formatted url
def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_1'
    search_term = search_term.replace(' ','+')
    url = template.format(search_term)
    url = url + '&page={}'
    return url

def extract_item(item):
    atag = item.h2.a
    description = atag.text.strip()
    
    link = 'https://www.amazon.com' + atag.get('href')
    
    try:
        price = item.find('span','a-price')
        price_item = price.find('span','a-offscreen').text
    except AttributeError:
        return
    
    try: 
        rating = item.i.text
        review_count = item.find('span',{'class':'a-size-base','dir':'auto'}).text
    except AttributeError:
        rating=''
        review_count=''
        
    result = (description, price_item, rating, review_count, link)
    return result

def main(search_term):
    browser = webdriver.Chrome(executable_path='C:/Users/HP/chromedriver.exe')
    url = get_url(search_term)
    records = []
    for page_num in range(1, 15):
        browser.get(url.format(page_num))
        soup = bs(browser.page_source, 'lxml')
        items = soup.find_all('div',{'data-component-type':'s-search-result'})
        
        for item in items:
            result = extract_item(item)
            if result:
                records.append(result)
                
    browser.close()
    
    with open('results.csv', 'w',newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description','Price','Rating','ReviewCount','Url'])
        writer.writerows(records)

In [102]:
main('Gaming Laptops')